# Diseño de Filtros y Comparación entre IIR y FIR

En este cuaderno, diseñaremos y compararemos diferentes tipos de filtros de la familia IIR (Butterworth, Chebyshev I, Chebyshev II, Bessel, Elíptico) y un filtro FIR diseñado con la función `firwin` de SciPy. Además, crearemos un cuadro comparativo de las diferencias entre filtros IIR y FIR.

## Paso 1: Descargar 10 segundos de una canción
Usa herramientas como `youtube-dl` o cualquier otra herramienta online para descargar una canción desde YouTube. Luego recorta los primeros 10 segundos usando una herramienta como `pydub`.

In [ ]:
from pydub import AudioSegment

# Cargar el archivo de audio descargado
audio = AudioSegment.from_file("cancion.mp3")

# Recortar los primeros 10 segundos
audio_10_segundos = audio[:10000]  # 10000 ms = 10 segundos

# Guardar el fragmento de 10 segundos
audio_10_segundos.export("cancion_10_segundos.wav", format="wav")

## Paso 2: Diseño de Filtros IIR (Butterworth, Chebyshev I, Chebyshev II, Bessel, Elíptico)
Implementamos filtros IIR con la biblioteca `scipy.signal` y comparamos su respuesta en frecuencia.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, cheby1, cheby2, bessel, ellip, lfilter, freqz

# Función para diseñar filtros IIR
def iir_filter_design(filter_type, order, lowcut, highcut, fs, rp=None, rs=None):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    
    if filter_type == 'butterworth':
        b, a = butter(order, [low, high], btype='band')
    elif filter_type == 'chebyshev1':
        b, a = cheby1(order, rp, [low, high], btype='band')
    elif filter_type == 'chebyshev2':
        b, a = cheby2(order, rs, [low, high], btype='band')
    elif filter_type == 'elliptic':
        b, a = ellip(order, rp, rs, [low, high], btype='band')
    elif filter_type == 'bessel':
        b, a = bessel(order, [low, high], btype='band', analog=False)
    
    return b, a

### Parámetros para el filtro
- `order`: Orden del filtro.
- `lowcut`: Frecuencia de corte baja.
- `highcut`: Frecuencia de corte alta.
- `fs`: Frecuencia de muestreo.
- `rp`: Para Chebyshev I y Elíptico, define el máximo ripple en la banda de paso (en dB).
- `rs`: Para Chebyshev II y Elíptico, define la atenuación mínima en la banda de rechazo (en dB).

In [ ]:
# Función para graficar respuestas en frecuencia
def plot_filter_responses(filters, fs):
    plt.figure(figsize=(10, 8))
    
    for f_type, (b, a) in filters.items():
        w, h = freqz(b, a, worN=2000)
        plt.plot((fs * 0.5 / np.pi) * w, np.abs(h), label=f_type)

    plt.title('Respuesta de frecuencia de los filtros')
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('Ganancia')
    plt.grid()
    plt.legend()
    plt.show()

# Frecuencia de muestreo y frecuencias de corte
fs = 5000
lowcut = 500
highcut = 1500

# Orden del filtro
order = 4

# Diseño de los filtros
filters = {
    'butterworth': iir_filter_design('butterworth', order, lowcut, highcut, fs),
    'chebyshev1': iir_filter_design('chebyshev1', order, lowcut, highcut, fs, rp=1),
    'chebyshev2': iir_filter_design('chebyshev2', order, lowcut, highcut, fs, rs=40),
    'elliptic': iir_filter_design('elliptic', order, lowcut, highcut, fs, rp=1, rs=40),
    'bessel': iir_filter_design('bessel', order, lowcut, highcut, fs)
}

# Graficar la respuesta en frecuencia de cada filtro
plot_filter_responses(filters, fs)

## Paso 3: Método de diseño de filtros FIR por ventaneo (firwin)
Implementamos un filtro FIR pasabanda utilizando el método de ventaneo.

In [ ]:
from scipy.signal import firwin

# Función para diseñar un filtro FIR usando firwin
def fir_filter_design(numtaps, lowcut, highcut, fs, window='hamming'):
    nyquist = 0.5 * fs
    taps = firwin(numtaps, [lowcut / nyquist, highcut / nyquist], pass_zero=False, window=window)
    return taps

# Diseñar un filtro FIR pasabanda
numtaps = 101  # Número de coeficientes del filtro
taps = fir_filter_design(numtaps, lowcut, highcut, fs)

# Graficar la respuesta en frecuencia
w, h = freqz(taps, worN=2000)
plt.plot((fs * 0.5 / np.pi) * w, np.abs(h), label='FIR (Hamming)')
plt.title('Respuesta de frecuencia del filtro FIR')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Ganancia')
plt.grid()
plt.legend()
plt.show()

## Paso 4: Cuadro comparativo de filtros IIR y FIR
| **Característica**           | **Filtros IIR**                           | **Filtros FIR**                            |
|------------------------------|-------------------------------------------|--------------------------------------------|
| **Estabilidad**               | Pueden ser inestables debido a polos en el plano \( z \). | Siempre estables. |
| **Fase**                     | Fase no lineal, lo que puede distorsionar la señal. | Fase lineal si se diseña adecuadamente. |
| **Complejidad computacional** | Menor orden que FIR para especificaciones similares. | Requieren un orden mayor para lograr la misma atenuación en banda de rechazo. |
| **Respuesta impulsiva**       | Infinita (responde indefinidamente).      | Finita (se apaga después de cierto tiempo). |
| **Aplicaciones**              | Procesamiento en tiempo real donde se requiere una baja latencia. | Aplicaciones donde la distorsión de fase no es aceptable, como audio e imagen. |